In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from tqdm import tqdm
#from classic_network import LeNet_5, AlexNet, VGG16
%matplotlib inline

In [2]:
tf.__version__

'2.0.0-alpha0'

# LeNet-5

In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

def prepros(x, y, INPUT_SIZE=(32, 32, 1)):
    N = y.size
    x = np.pad(x, ((0,0),(2,2),(2,2)), 'constant').reshape((N, *INPUT_SIZE))
    y_ = np.zeros((N, 10))
    y_[np.arange(N), y] = 1
    return x, y_

x_train, y_train = prepros(x_train, y_train)
x_test, y_test = prepros(x_test, y_test)

In [4]:
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Activation
from layers import SubPooling

A = tf.constant(1.7159)
S = tf.constant(2/3)

LeNet_5 = tf.keras.models.Sequential([
    Conv2D(name='C1',
           filters=6,
           kernel_size=(5, 5),
           strides=(1, 1),
           padding='valid',
           input_shape=(32, 32, 1)),

    SubPooling(name='S2',
               features=6),
    
    Conv2D(name='C3',
           filters=16,
           kernel_size=(5, 5),
           strides=(1, 1),
           padding='valid'),
    
    SubPooling(name='S4',
               features=16),
    
    Conv2D(name='C5',
           filters=120,
           kernel_size=(5, 5),
           strides=(1, 1),
           padding='valid'),
    
    Flatten(name='F5'),
    
    Dense(name='F6',
          units=84,
          activation=Activation(lambda x: A * tf.nn.tanh(S*x))),
    
    Dense(name='F7',
          units=10,
          activation='softmax')
        
], name='LeNet5')

LeNet_5.summary()

Model: "LeNet5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
C1 (Conv2D)                  (None, 28, 28, 6)         156       
_________________________________________________________________
S2 (SubPooling)              (None, 14, 14, 6)         12        
_________________________________________________________________
C3 (Conv2D)                  (None, 10, 10, 16)        2416      
_________________________________________________________________
S4 (SubPooling)              (None, 5, 5, 16)          32        
_________________________________________________________________
C5 (Conv2D)                  (None, 1, 1, 120)         48120     
_________________________________________________________________
F5 (Flatten)                 (None, 120)               0         
_________________________________________________________________
F6 (Dense)                   (None, 84)                10164

In [5]:
LeNet_5.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [6]:
BATCH_SIZE = 16

LeNet_5.fit(x_train, y_train,
            batch_size=BATCH_SIZE,
            validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
60000/60000 [==============================] - 51s 842us/sample - loss: 0.2593 - accuracy: 0.9164 - val_loss: 0.1369 - val_accuracy: 0.9548


# AlexNet

In [3]:
from utils import read_image_dataset

TRAIN = 'imagenet12/train/'
TEST = 'imagenet12/test/'
VAL = 'imagenet12/val/'

train, N = read_image_dataset(TRAIN)

In [4]:
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout

AlexNet = tf.keras.models.Sequential([
    Conv2D(filters=96,
           kernel_size=(11, 11),
           strides=(4, 4),
           padding='valid',
           activation='relu',
           input_shape=(227, 227, 3)),
    
    MaxPool2D(pool_size=(3, 3),
              strides=(2, 2),
              padding='valid'),
    
    Conv2D(filters=256,
           kernel_size=(5, 5),
           strides=(1, 1),
           padding='same',
           activation='relu'),
    
    MaxPool2D(pool_size=(3, 3),
              strides=(2, 2),
              padding='valid'),
    
    Conv2D(filters=384,
           kernel_size=(3, 3),
           strides=(1, 1),
           padding='same',
           activation='relu'),
    
    Conv2D(filters=384,
           kernel_size=(3, 3),
           strides=(1, 1),
           padding='same',
           activation='relu'),
    
    Conv2D(filters=256,
           kernel_size=(3, 3),
           strides=(1, 1),
           padding='same',
           activation='relu'),    
    
    MaxPool2D(pool_size=(3, 3),
              strides=(2, 2),
              padding='valid'),
    
    Flatten(),
    
    Dense(4096, activation='relu'),
    Dropout(0.5),
    Dense(4096, activation='relu'),
    Dropout(0.5),
    Dense(1000, activation='softmax')
])
AlexNet.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 55, 55, 96)        34944     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 27, 27, 256)       614656    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 256)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 13, 384)       885120    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 13, 13, 384)       1327488   
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 13, 13, 256)       8

In [5]:
WD = 5e-4 / 2
BS = 128
Mom = 0.9
LR = 1e-2
EPOCHS = 10

In [6]:
regularizer = tf.keras.regularizers.l2(WD)
for layer in AlexNet.layers:
    if hasattr(layer, 'kernel_regularizer'):
        setattr(layer, 'kernel_regularizer', regularizer)

In [7]:
sgd = tf.keras.optimizers.SGD(lr=LR, momentum=Mom)
AlexNet.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
tensorboard = tf.keras.callbacks.TensorBoard(log_dir='logs/alexnet')

In [11]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train = train.shuffle(buffer_size=N)
train = train.batch(BS).prefetch(buffer_size=AUTOTUNE)
SPE =tf.math.ceil(N/BS)

In [ ]:
AlexNet.fit(train, epochs=EPOCHS, steps_per_epoch=SPE, callbacks=[tensorboard])
tf.keras.models.save_model(AlexNet, 'alexnet.h5')

Epoch 1/10


# VGG16

In [11]:
tf.reset_default_graph()

In [12]:
# TODO: Crop ImageNet

In [13]:
x = tf.placeholder(tf.float32, shape=[None, 224, 224, 3])
y = tf.placeholder(tf.int32, shape=[None])
one_hot_y = tf.one_hot(y, 1000)

In [14]:
BS = 256
Mom = 0.9
LR = 1e-2
logits = VGG16(x)
cost = tf.losses.softmax_cross_entropy(one_hot_y, logits)
optimizer = tf.train.MomentumOptimizer(learning_rate=LR,
                                       momentum=Mom)
training = optimizer.minimize(cost)

# ResNet

In [7]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train = x_train / 255
x_test = x_test / 255
y_train = y_train.flatten()
y_test = y_test.flatten()
print(x_train.shape)

(50000, 32, 32, 3)


In [8]:
tf.reset_default_graph()

x = tf.placeholder(tf.float32, shape=[None, 32, 32, 3])
y = tf.placeholder(tf.int64, shape=[None])
one_hot_y = tf.one_hot(y, 10)

In [9]:
from resnet import ResBlock

w1 = tf.Variable(tf.truncated_normal([3, 3, 3, 16], stddev=tf.sqrt(2/(3 * 3 * 3))))
w2 = tf.Variable(tf.truncated_normal([1024, 10], stddev=tf.sqrt(2/1024)))
b1 = tf.Variable(tf.zeros(16))
b2 = tf.Variable(tf.zeros(10))

z1 = tf.nn.conv2d(input=x, filter=w1, strides=[1, 1, 1, 1], padding='SAME') + b1
a1 = tf.nn.relu(tf.layers.batch_normalization(z1))

#r2 = ResBlock(ResBlock(ResBlock(a1)))
#r3 = ResBlock(ResBlock(ResBlock(r2, bottleneck=True)))
#r4 = ResBlock(ResBlock(ResBlock(r3, bottleneck=True)))


w2T = tf.Variable(tf.truncated_normal([32, 32, 16, 16], stddev=tf.sqrt(2/(32 * 32 * 16))))
b2T = tf.Variable(tf.zeros(16))

z2 = tf.nn.conv2d(input=a1, filter=w2T, strides=[1, 1, 1, 1], padding='SAME') + b2T
a2 = tf.nn.relu(tf.layers.batch_normalization(z2))

#r2T = ResBlock(a1, plain=True)
#s3T = tf.nn.max_pool(r2T, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
#r4T = ResBlock(ResBlock(s3T, plain=True), plain=True)
#s5T = tf.nn.max_pool(r4T, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')



s5T = tf.nn.max_pool(a2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
#f5 = tf.reshape(s5T, [-1, 16 * 16 * 16])

inp = 16 * 16 * 16
f5 = tf.reshape(s5T, [-1, inp])

w2T = tf.Variable(tf.truncated_normal([inp, 1024], stddev=tf.sqrt(2/inp)))
b2T = tf.Variable(tf.zeros(1024))
f6 = tf.nn.relu(tf.layers.batch_normalization(tf.nn.dropout(tf.matmul(f5, w2T) + b2T, keep_prob=0.5)))

w3T = tf.Variable(tf.truncated_normal([1024, 10], stddev=tf.sqrt(2/1024)))
b3T = tf.Variable(tf.zeros(10))
logits = tf.nn.softmax(tf.matmul(f6, w3T) + b3T)

#r3T = ResBlock(ResBlock(a1, plain=True), plain=True)

#s5 = tf.nn.avg_pool(r4, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
#f5 = tf.reshape(s5, [-1, 4 * 4 * 64])
#f6 = tf.matmul(f5, w2) + b2
#logits = tf.nn.softmax(f6)
#s5

In [10]:
LR = 0.001
WD = 0.0001
M = 0.9
BATCH_SIZE = 32
EPOCHS = 10

pred = tf.argmax(logits, axis=-1)
cross_entropy = tf.losses.softmax_cross_entropy(onehot_labels=one_hot_y,
                                                logits=logits)
loss = tf.reduce_mean(cross_entropy)
optimizer = tf.contrib.opt.MomentumWOptimizer(weight_decay=WD, learning_rate=LR, momentum=M)
train_op = optimizer.minimize(loss)

correct_pred = tf.equal(pred, y)
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32)) * 100

tf.summary.scalar('acc_summary', accuracy)
merged = tf.summary.merge_all()

In [11]:
STORE_PATH = 'logs/'
saver = tf.train.Saver()
ws = {}
N = 3125
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter(STORE_PATH, sess.graph)

    for i in range(EPOCHS):
        x_train, y_train = shuffle(x_train, y_train)
        for offset in tqdm(range(0, y_train.size, BATCH_SIZE)):
            end = offset + BATCH_SIZE
            xt, yt = x_train[offset:end], y_train[offset:end]
            sess.run(train_op, feed_dict={x: xt, y: yt})
            
        loss_v, acc, summary = sess.run([loss, accuracy, merged], feed_dict={x: xt, y: yt})
        print("Epoach {}, Minibatch Loss= {:.4f}, Training Accuracy={:.2f}%".format(i, loss_v, acc))
        writer.add_summary(summary, i)

    print("Optimization Finished!")
    save_path = saver.save(sess, "logs/resnet.ckpt")

    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: x_test,
                                      y: y_test}))
    ws = sess.run(params)

  0%|          | 4/1563 [00:23<2:35:19,  5.98s/it]

KeyboardInterrupt: 